## This notebook trains a Simple Neural Network to predict if a patient has Diabetes or not

In [11]:
#Import the Libraries
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import pandas as pd
import torch


## Read the Data and Convert it to Numpy

In [2]:
#Just a comment added
diabetes_data = pd.read_csv("diabetes.csv")
diabetes_data.head()
sc_X = StandardScaler()
X =  pd.DataFrame(sc_X.fit_transform(diabetes_data.drop(["Outcome"],axis = 1),),
        columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'])


X = X.to_numpy()
Y = diabetes_data["Outcome"]
Y = Y.to_numpy()
Y = np.array([Y])
Y = Y.reshape((768,1))


## Prepare a Pytorch DataLoader 

In [3]:

class  Training_DataSet(Dataset):
    def __init__(self,x,y,root_dir,transform):
        self.x = x
        self.y = y
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self,idx):
        features = self.x[idx]  
        target = self.y[idx]
        
        sample = {"features":features,"target":target}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
    

# To tensor class
class to_tensor_training(object):
    def __call__(self,sample):
        features , target = sample['features'],sample['target']
        features = np.array([features])
        features = features.astype('float32')
        features = torch.from_numpy(features)
        features = features.float()
        feeatues = features.squeeze(0)
        target = np.array([target])
        target = torch.from_numpy(target)
        target = target.long()
        
        return {"features":features,"target":target}

In [4]:
transformed_dataset = Training_DataSet(x=X,
                                       y=Y,
                                       root_dir="/Diabetes",
                                       transform = to_tensor_training())

In [5]:
#Print some Data
for i in range(len(transformed_dataset)):
    sample = transformed_dataset[i]

    print(i, sample['features'].shape,sample['target'].shape)

    if i == 3:
        break

0 torch.Size([1, 8]) torch.Size([1, 1])
1 torch.Size([1, 8]) torch.Size([1, 1])
2 torch.Size([1, 8]) torch.Size([1, 1])
3 torch.Size([1, 8]) torch.Size([1, 1])


In [6]:
#Create a simple Neural Network
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.fc1 = nn.Linear(8,128)
        self.fc2 = nn.Linear(128,2)
        
    def forward(self,x):
        #drouput_activation1 = nn.Dropout(p=0.3)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x
        

In [7]:
net = Net()

In [8]:
trainloader = torch.utils.data.DataLoader(transformed_dataset,batch_size=4, shuffle=True,num_workers=2)


In [9]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Train the Network

In [10]:
loss_array = []
correct = 0
total = 0

accuracy_array = []
for epoch in range(11):
    running_loss = 0
    for i , data in enumerate(trainloader,0):
        features , target = data['features'],data['target']
        target = target.squeeze()
        optimizer.zero_grad()
        outputs = net(features)
        outputs = outputs.squeeze()
        _, predictions = torch.max(outputs,1)
        total += target.size(0)
        correct += (predictions == target).sum().item()
        loss = criterion(outputs,target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        
    loss_array.append(running_loss / len(trainloader))
    
    print("Epoch: {}/ 10.. ".format(epoch+1),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)))
    print("Accuracy  %d %%" %(100*correct/total))
    correct = 0
    total = 0
print("Finished training")

Epoch: 1/ 10..  Training Loss: 0.578.. 
Accuracy  70 %
Epoch: 2/ 10..  Training Loss: 0.487.. 
Accuracy  76 %
Epoch: 3/ 10..  Training Loss: 0.472.. 
Accuracy  76 %
Epoch: 4/ 10..  Training Loss: 0.467.. 
Accuracy  77 %
Epoch: 5/ 10..  Training Loss: 0.462.. 
Accuracy  76 %
Epoch: 6/ 10..  Training Loss: 0.459.. 
Accuracy  77 %
Epoch: 7/ 10..  Training Loss: 0.457.. 
Accuracy  78 %
Epoch: 8/ 10..  Training Loss: 0.453.. 
Accuracy  77 %
Epoch: 9/ 10..  Training Loss: 0.454.. 
Accuracy  78 %
Epoch: 10/ 10..  Training Loss: 0.448.. 
Accuracy  77 %
Epoch: 11/ 10..  Training Loss: 0.448.. 
Accuracy  78 %
Finished training
